In [3]:
import numpy as np
import pandas as pd

import requests

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

import os

from sklearn.model_selection import train_test_split

import time
from pathlib import Path

from PIL import Image
from io import BytesIO

In [4]:
# try not to include user directories
# data_path = Path(r"C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\data")
# ./data added to gitignore
data_path = Path(r"./data")
data_path.mkdir(parents=True, exist_ok=True)

bee_data_path = data_path / "busy_bee.csv"

bee_data = pd.read_csv(bee_data_path)

FileNotFoundError: [Errno 2] No such file or directory: 'data\\busy_bee.csv'

In [5]:
bee_data.head()

NameError: name 'bee_data' is not defined

In [ ]:
bee_data.count()

In [ ]:
bee_genuses = bee_data.taxon_genus_name

plt.figure(figsize=(20,6))
plt.bar(bee_genuses.value_counts().keys(), bee_genuses.value_counts().values)
plt.ylabel("Count")

In [ ]:
bee_genus_distribution = plt.pie(bee_genuses.value_counts(ascending=False, normalize=True)[:5])
plt.show()
bee_genuses.value_counts(ascending=False, normalize=True)[:5]

In [ ]:
bee_taxons = bee_data.taxon_tribe_name

plt.figure(figsize=(20,6))
plt.bar(bee_taxons.value_counts().keys(), bee_taxons.value_counts().values, color="red")
plt.ylabel("Count")

In [ ]:
bee_taxon_distribution = plt.pie(bee_taxons.value_counts(ascending=False, normalize=True)[:5])
plt.show()
bee_taxons.value_counts(ascending=False, normalize=True)[:5]

In [ ]:
missing_images = bee_data.image_url.isnull().sum()               # checks to see if there are any missing images
missing_tribe_name = bee_data.taxon_tribe_name.isnull().sum()    # checks to see if there are any missing tribes

print(f"Missing Bee Images: {missing_images}")
print(f"Missing Tribe Images: {missing_tribe_name}")

In [ ]:
bee_data = bee_data[~bee_data.taxon_tribe_name.isnull()]         # Filters the dataset that does not include missing tribe names

In [ ]:
bee_data.head()

In [ ]:
def show_bee_img(img: Image):
    """
    Shows an image of a Bee

    Args:
        img (Image): PIL Image format
    """
    
    plt.figure(figsize=(10,10))
    plt.axis("off")
    plt.imshow(img)

In [ ]:
def scrape_bee_image(url_series:pd.Series) -> list:
    """
    Iterates through the url series and requests these url links to get the images
    Uses requests to get image content and PIL to convert the request content into an Image

    Args:
        url_series (pd.Series): a Pandas Series that includes all the urls for the pictures

    Returns:
        list: a list of PIL Image that can be shown using matplotlib show() method
    """
    
    total_img = len(url_series)
    bee_images = []
    failed_requests = []
    time_stats = []
    
    epoch_div_time = time.time()
    start_time = time.time()
    for i in range(total_img): # set the 1000 for now, should be the size of the entire dataset
        epoch_time = time.time()
        try:
            url = url_series.iloc[i]

            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            
            bee_images.append(img)
            time_stats.append(time.time() - epoch_time)
            
            if i % 1000 == 0:
                print(f"Image {i}: Total Run Time -> {time.time() - epoch_div_time}")
                epoch_div_time = time.time()
                
        except:
            failed_requests.append(i)
            print("Error 404: Failure to Access Image URL")
    
    print(f"\nTotal Time in Minutes {(time.time() - start_time) / 60}")
    return bee_images, time_stats, failed_requests

In [ ]:
def vizualize_images(image_set: list, image_labels:pd.Series, plot_dim: tuple=(3,3), seed: int=42) -> None:
    """
    Plots the images into a grid system
    Each image's title is their label

    Args:
        image_set (list): = a list containing all the images in PIL Image format
        image_labels (_type_): the labels representing the image_set
        plot_dim (tuple, optional): the dimensions of the image grid. Defaults to (3,3).
        seed (int, optional): sets the seed for the random number generator. Defaults to 42.
    """
    
    height, width = plot_dim
    fig, ax = plt.subplots(height, width, figsize=(16,8))
    
    np.random.seed(seed)

    for i in range(height):
        for j in range(width):
            m = len(image_set)
            random_num = np.random.randint(0, m)

            img = image_set[random_num]
            img_label = image_labels.iloc[random_num]

            ax[i, j].set_title(img_label)
            ax[i, j].imshow(img)
            ax[i, j].axis(False)

In [ ]:
bee_url_images = bee_data.image_url
bee_images, time_stats, failed_image_indexes = scrape_bee_image(bee_url_images)

In [ ]:
print(len(bee_images))

In [ ]:
print(len(y.drop(index=failed_image_indexes)))

In [ ]:
y = bee_data.taxon_tribe_name # labels of our dataset is going to be the bee_taxon_tribe values

In [ ]:
vizualize_images(bee_images, y.drop(index=failed_image_indexes)) 

In [ ]:
# Plots how much each epochs takes to acquire the image
plt.figure(figsize=(15,6))
time_data = sns.scatterplot(x=[i for i, _ in enumerate(time_stats)], y=time_stats, color="green")
time_data.set(ylabel="Time to Request Image (sec)", xlabel="EPOCHS")

In [ ]:
#========= Testing ===========#
url = bee_data.image_url.iloc[0]

# playing around with getting pictures using url link
try:
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
except:
    print("Failure to access link")
    
#========= Testing ===========#

In [ ]:
#========= Testing ===========#

# playing around with os library
url = bee_data.image_url.iloc[0]

response = requests.get(url)
image = Image.open(BytesIO(response.content))


if not os.path.exists("bee_data"):
    os.makedirs("bee_data")

# Save the image to the new directory
# image.save(os.path.join("bee_data", "bee.jpg"))

# Create two categories, training, and testing,
# From those catagories create the twenty two different tribe folders with each bee photo
# The bee is then classified into different folders 

#========= Testing ===========#

In [ ]:
classification_folders = y.unique()

file_structure = classification_folders

file_structure

In [ ]:
simplified_df = pd.DataFrame({"Bee_Images": bee_images, "Taxon_Tribe": y.drop(index=failed_image_indexes)})

In [ ]:
def generate_dir(parent_dir:str, file_structure:dict) -> None:
    """
    Generates the directory structure to store the train and test data from our bee dataset

    Args:
        parent_dir (str): The path location of the parent directory
        file_structure (dict): a dictionary containing file structure 
        
    file_structure Example:
        
        {'data': ['Xylocopini', 'Bombini', 'Apini', 'Ammobatoidini', 'Megachilini',
                   'Eucerini', 'Emphorini', 'Anthidiini', 'Halictini', 'Anthophorini',
                   'Osmiini', 'Augochlorini', 'Sphecodini', 'Nomadini', 'Ceratinini',
                    nan, 'Epeolini', 'Protandrenini', 'Andrenini', 'Calliopsini',
                   'Panurgini', 'Caupolicanini', 'Melectini']),
        }
        
    """
    
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    
    for key in file_structure.keys():
        if not os.path.exists(parent_dir + f"/{key}"):
            os.makedirs(parent_dir + f"/{key}")
            
        train_test_path = os.path.join(parent_dir, key)
        
        for folder in file_structure[key]:
            
            if not os.path.exists(train_test_path + f"/{folder}"):
                os.makedirs(train_test_path + f"/{folder}")
            
        

In [ ]:
def image_distribution(parent_dir:str, data:pd.DataFrame) -> None:
    """
    Classifies the images into their unique classification folder 

    Args:
        parent_dir (str): The path location of the parent directory
        train_data (pd.DataFrame): Dataset for the training set
        test_data (pd.DataFrame): Dataset for the testing set
    """
    
    phases = os.listdir(parent_dir) 
    
    train_path = os.path.join(parent_dir, phases[1])
    test_path = os.path.join(parent_dir, phases[0]) 
    
    # You can probably do recursion for this to make it look nicer
    
    for phase in os.listdir(parent_dir):
        
        if phase == "train":
            for i in range(len(data)):
                # These may need to be changed depending on your label column name
                folder_type = train_data.iloc[i]["Taxon_Tribe"] 
                
                # These may need to be changed depending on your image column name
                image = train_data.iloc[i]["Bee_Images"]
                
                image_save_path = train_path + f"/{folder_type}"
                try:
                    image.save(os.path.join(image_save_path, f"{folder_type}_{i}.jpg"))
                
                except:
                    print("cannot write mode RGBA as JPEG")
                print(f"Image {i} saved to {image_save_path}")    

# The Section is commented below for testing purposes

#         if phase == "test":
#             for i in range(len(train_data)):
#                 folder_type = test_data.iloc[i]["Taxon_Tribe"]
#                 image = test_data.iloc[i]["Bee_Images"]
                
#                 image_save_path = test_path + f"/{folder_type}"
#                 image.save(os.path.join(image_save_path, f"{folder_type}_{i}.jpg"))

In [ ]:
# im = Image.open("audacious.png")
# rgb_im = im.convert('RGB')
# rgb_im.save('audacious.jpg')

# Future solving rn but my brain rn is fried

In [ ]:
X = simplified_df["Bee_Images"]
y = simplified_df["Taxon_Tribe"]

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.6, test_size=0.4, random_state=42)

train_bee_data = pd.concat([X_train, y_train], axis=1)
test_bee_data = pd.concat([X_test, y_test], axis=1)

In [ ]:
len(y.unique())

In [ ]:
sorted(train_bee_data["Taxon_Tribe"].unique())

In [ ]:
sorted(test_bee_data["Taxon_Tribe"].unique())

In [ ]:
# set our parent directory to whatever location you want
parent_dir = r"C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data"
generate_dir(parent_dir, file_structure)

In [ ]:
image_distribution(parent_dir, simplified_df)

Image 93 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 94 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 95 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 96 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 97 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 98 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 99 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Emphorini
Image 100 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 101 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Le

Image 167 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 168 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 169 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 170 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 171 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 172 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 173 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 174 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 175 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Lea

Image 247 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 248 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 249 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 250 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 251 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 252 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 253 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 254 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 255 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Le

Image 331 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 332 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 333 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 334 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 335 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 336 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 337 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 338 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Megachilini
Image 339 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Lea

Image 412 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 413 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 414 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 415 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Halictini
Image 416 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 417 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 418 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 419 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 420 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Ma

...

Image 10860 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Calliopsini
Image 10861 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 10862 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10863 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10864 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 10865 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10866 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10867 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 10868 saved to C:\Users\Brand\Documents\Branden's Stuff

Image 10932 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 10933 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 10934 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10935 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10936 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10937 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 10938 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 10939 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 10940 saved to C:\Users\Brand\Documents\Branden's Stuff\

Image 11006 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11007 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11008 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11009 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Halictini
Image 11010 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 11011 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 11012 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11013 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11014 saved to C:\Users\Brand\Documents\Branden's Stuff\Pyth

Image 11080 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11081 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
cannot write mode RGBA as JPEG
Image 11082 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11083 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 11084 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Anthidiini
Image 11085 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 11086 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11087 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11088 saved to C:\Users\Bran

Image 11155 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11156 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 11157 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Apini
Image 11158 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11159 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11160 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Xylocopini
Image 11161 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Emphorini
Image 11162 saved to C:\Users\Brand\Documents\Branden's Stuff\Python\Machine Learning\PyTorch\bee_data\train/Bombini
Image 11163 saved to C:\Users\Brand\Documents\Branden's Stuff\P

In [ ]:
for dirpath, dirnames, filenames in os.walk(parent_dir):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")